In [211]:
import numpy as np
import pandas as pd
from hmmlearn.hmm import GaussianHMM
import plotly.graph_objects as go
from plotly.graph_objs.scatter.marker import Line
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture
import math
import warnings
warnings.filterwarnings('ignore')

In [212]:
price_data = pd.read_csv('SP500_E-mini_Futures_CME.csv')
price_data

,time,date,open,high,low,close,Volume
0,873756000,1997-08-09,933.75,941.25,932.75,934.00,11387
1,873842400,1997-09-09,933.75,934.25,914.50,915.25,2523
2,873928800,1997-10-09,924.00,929.75,910.25,918.25,9945
3,874015200,1997-11-09,918.00,936.50,913.50,933.75,8911
4,874274400,1997-14-09,933.50,939.75,929.00,931.50,11874
...,...,...,...,...,...,...,...
6595,1696197600,2023-01-10,4349.00,4355.50,4295.50,4324.25,1896276
6596,1696284000,2023-02-10,4326.00,4335.75,4251.25,4264.75,2192628
6597,1696370400,2023-03-10,4262.75,4304.00,4235.50,4297.75,2098919
6598,1696456800,2023-04-10,4291.50,4302.00,4258.00,4290.75,1625316


In [213]:
trading_instrument = 'ESc1'
prices = price_data[['date','close']].copy()
prices.rename(columns={'close':f'{trading_instrument}'},inplace=True)
prices = prices.set_index('date')
prices.index = pd.to_datetime(prices.index,format="%Y-%d-%m")
prices

,ESc1
date,
1997-09-08,934.00
1997-09-09,915.25
1997-09-10,918.25
1997-09-11,933.75
1997-09-14,931.50
...,...
2023-10-01,4324.25
2023-10-02,4264.75
2023-10-03,4297.75


In [214]:
prices.columns.name = trading_instrument
px.line(prices[f'{trading_instrument}'])

Data Engineering

In [215]:
def prepare_data_for_model_input(prices, ma):
    '''
        Input:
        prices (df) - Dataframe of close prices
        ma (list) - list of moving average lengths
        
        Output:
        prices(df) - An enhanced prices dataframe, with moving averages and log return columns
        prices_array(nd.array) - an array of log returns
    '''
    
    intrument = prices.columns.name
    prices[f'{intrument}_ma'] = prices.rolling(ma).mean()
    prices[f'{intrument}_log_return'] = np.log(prices[f'{intrument}_ma']/prices[f'{intrument}_ma'].shift(1)).dropna()
 
    prices.dropna(inplace = True)
    prices_array = np.array([[q] for q in prices[f'{intrument}_log_return'].values])
    
    return prices, prices_array

In [216]:
prices, prices_array = prepare_data_for_model_input(prices, 7)
prices

ESc1,ESc1,ESc1_ma,ESc1_log_return
date,,,
1997-09-17,957.75,938.357143,0.003622
1997-09-18,960.50,944.821429,0.006865
1997-09-21,966.25,951.678571,0.007231
1997-09-22,962.00,955.714286,0.004232
1997-09-23,954.50,959.000000,0.003432
...,...,...,...
2023-10-01,4324.25,4336.464286,-0.001572
2023-10-02,4264.75,4322.714286,-0.003176
2023-10-03,4297.75,4311.142857,-0.002680


In [217]:
prices_array

array([[ 0.00362229],
       [ 0.00686532],
       [ 0.0072314 ],
       ...,
       [-0.00268048],
       [-0.0007956 ],
       [ 0.00092814]])

Modelling and in-sample testing

In [218]:
class RegimeDetection:
 
    def get_regimes_hmm(self, input_data, params):
        hmm_model = self.initialise_model(GaussianHMM(), params).fit(input_data)
        return hmm_model
    
    def get_regimes_clustering(self, params):
        clustering =  self.initialise_model(AgglomerativeClustering(), params)
        return clustering
        
    def initialise_model(self, model, params):
        for parameter, value in params.items():
            setattr(model, parameter, value)
        return model

In [219]:
def plot_hidden_states(hidden_states, prices_df):
    
    '''
    Input:
    hidden_states(numpy.ndarray) - array of predicted hidden states
    prices_df(df) - dataframe of close prices
    
    Output:
    Graph showing hidden states and prices
    
    '''
    
    colors = ['blue', 'green']
    n_components = len(np.unique(hidden_states))
    fig = go.Figure()
 
    for i in range(n_components):
        mask = hidden_states == i
        print('Number of observations for State ', i,":", len(prices_df.index[mask]))
        
        fig.add_trace(go.Scatter(x=prices_df.index[mask], y=prices_df[f"{prices_df.columns.name}"][mask],
                    mode='markers',  name='Hidden State ' + str(i), marker=dict(size=4,color=colors[i])))
        
    fig.update_layout(height=400, width=900, legend=dict(
            yanchor="top", y=0.99, xanchor="left",x=0.01), margin=dict(l=20, r=20, t=20, b=20)).show()

In [220]:
regime_detection = RegimeDetection()

In [221]:
params = {'n_clusters': 2, 'linkage': 'complete', 'metric': 'manhattan', 'random_state':100}
clustering = regime_detection.get_regimes_clustering(params)
clustering_states = clustering.fit_predict(prices_array)
 
plot_hidden_states(np.array(clustering_states), prices[[f'{trading_instrument}']])

Number of observations for State  0 : 6464
Number of observations for State  1 : 129


In [222]:
params = {'n_components':2, 'covariance_type':"full", 'random_state':100}
 
hmm_model = regime_detection.get_regimes_hmm(prices_array, params)
hmm_states = hmm_model.predict(prices_array)
plot_hidden_states(np.array(hmm_states), prices[[f'{trading_instrument}']])

Number of observations for State  0 : 962
Number of observations for State  1 : 5631


Feed forward training and out of sample testing

In [223]:
def feed_forward_training(model, params, prices, split_index, retrain_step):
    '''
    Input:
    model (<class 'method'>) - either gmm (Gaussian Mixture Models) or hmm (Hidden Markov Model)
    params (dict) - dictionary of parameters for a model
    prices (df) - Dataframe of close prices
    split_index (str) - index to split initial traing dataset and out of sample testing set
    retrain_step (int) - number of observations after which we retrain the model
    
    Output:
    states_pred (numpy.ndarray) - array of predicted hidden states
    '''
    # train/test split and initial model training
    init_train_data = prices[:split_index]
    test_data = prices[split_index:]
    rd_model = model(init_train_data, params)
    
    # predict the state of the next observation
    states_pred = []
    for i in range(math.ceil(len(test_data))):
        split_index += 1
        preds = rd_model.predict(prices[:split_index]).tolist()
        states_pred.append(preds[-1])
        
        # retrain the existing model
        if i % retrain_step == 0:
            rd_model = model(prices[:split_index], params)
            
    return  states_pred

In [224]:
model_hmm =  regime_detection.get_regimes_hmm
params = {'n_components':2, 'covariance_type': 'full', 'random_state':100}
split_index = np.where(prices.index > '2006-01-01')[0][0]

In [225]:
states_pred_hmm = feed_forward_training(model_hmm, params, prices_array, split_index, 20)
plot_hidden_states(np.array(states_pred_hmm), prices[[f'{trading_instrument}']][split_index:])

Number of observations for State  0 : 364
Number of observations for State  1 : 4125


In [204]:
np.array(states_pred_hmm)

array([1, 1, 1, ..., 0, 0, 1])

Creating portfolio of trading strategies

In [226]:
def calculate_strat_returns(stock_df):
    '''
    Since all the functions for the strategies in this class have to calculate these returns, this function does it for them.
    Input dataframe stock_df
    Outputs stock_df with the return columns
    Note: stock_df must have 'signal' column of -1,0,1
    '''
    stock_df['log_return_buy_n_hold'] = np.log(stock_df[f'{trading_instrument}']).diff()

    # Makie it so you start calculating log_return_trend_follow after the first trade, ie. first time action != 0
    idx = (stock_df['action'] != 0)
    first_index = idx.idxmax()
    stock_df['log_return_strat'] = 0.0
    stock_df.loc[first_index:, 'log_return_strat'] = stock_df['log_return_buy_n_hold'] * stock_df['signal']

    stock_df['cum_return_buy_n_hold'] = np.exp(stock_df['log_return_buy_n_hold']).cumprod()
    stock_df['cum_return_strat'] = np.exp(stock_df['log_return_strat']).cumprod()
    
    return stock_df

In [261]:
class TradingStrategies:

    def sma_trend_following(self, input_data, sma1, sma2, strat_name, hmm = False, regime=1):
        '''
        Input:
        input_data (df) - dataframe of prices of instrument strategy will be done on
        sma (int) - number of sma lags
        signals (df) - dataframe of signals of hidden states
        
        Output:
        states_pred (df) - dataframe of signals
        '''
        # Calc SMA
        stock_df = input_data.copy()
        stock_df['SMA_'+str(sma1)] = stock_df[f'{trading_instrument}'].rolling(sma1).mean()
        stock_df['SMA_'+str(sma2)] = stock_df[f'{trading_instrument}'].rolling(sma2).mean()
        
        # Signals
        stock_df['signal'] = np.where(stock_df["SMA_"+str(sma1)] > stock_df["SMA_"+str(sma2)], 1, 0)
        stock_df['signal'] = np.where(stock_df["SMA_"+str(sma1)] < stock_df["SMA_"+str(sma2)], -1, stock_df['signal'])
        if hmm and regime == 1:
            stock_df['signal'] = np.where(stock_df['hmm'] == 0,0,stock_df['signal'])
        elif hmm and regime == 0:
            stock_df['signal'] = np.where(stock_df['hmm'] == 1,0,stock_df['signal'])       
        stock_df['action'] = stock_df.signal.diff()

        # Returns
        stock_df = calculate_strat_returns(stock_df)
        stock_df.columns.name = strat_name

        return stock_df

    def ewm_trend_following(self, input_data, a1, a2, strat_name, hmm = False, regime=1):
        # Calc EWM
        stock_df = input_data.copy()
        stock_df['EWM_'+str(a1)] = stock_df[f'{trading_instrument}'].ewm(alpha=a1, adjust=False).mean()
        stock_df['EWM_'+str(a2)] = stock_df[f'{trading_instrument}'].ewm(alpha=a2, adjust=False).mean()
        
        # Signals
        stock_df['signal'] = np.where(stock_df["EWM_"+str(a2)] > stock_df["EWM_"+str(a1)], 1, 0)
        stock_df['signal'] = np.where(stock_df["EWM_"+str(a2)] < stock_df["EWM_"+str(a1)], -1, stock_df['signal'])
        if hmm and regime == 1:
            stock_df['signal'] = np.where(stock_df['hmm'] == 0,0,stock_df['signal'])
        elif hmm and regime == 0:
            stock_df['signal'] = np.where(stock_df['hmm'] == 1,0,stock_df['signal'])  
        stock_df['action'] = stock_df.signal.diff()
        stock_df['action'] = stock_df['action'].where(stock_df['action'].isin([2, -2]), 0)

        # Returns
        stock_df = calculate_strat_returns(stock_df)
        stock_df.columns.name = strat_name

        return stock_df

trading_strategies = TradingStrategies() 


In [257]:
# Making a copy of the prices dataframe and using it as input for the trading strategies
trd_input = prices[[f'{trading_instrument}']][split_index:].copy()

trd_input['return'] = (trd_input[f'{trading_instrument}']/trd_input[f'{trading_instrument}'].shift(1))
trd_input['log'] = np.log(trd_input[f'{trading_instrument}'])
trd_input['log_return'] = np.log(trd_input['return'])
trd_input['hmm'] = np.array(states_pred_hmm) 
trd_input.dropna(inplace=True)

trd_input

ESc1,ESc1,return,log,log_return,hmm
date,,,,,
2006-01-03,1280.50,1.004511,7.155006,0.004501,1
2006-01-04,1281.25,1.000586,7.155591,0.000586,1
2006-01-05,1291.75,1.008195,7.163753,0.008162,1
2006-01-08,1295.00,1.002516,7.166266,0.002513,1
2006-01-09,1296.00,1.000772,7.167038,0.000772,1
...,...,...,...,...,...
2023-10-01,4324.25,0.999711,8.371994,-0.000289,0
2023-10-02,4264.75,0.986240,8.358139,-0.013855,0
2023-10-03,4297.75,1.007738,8.365847,0.007708,0


In [260]:
# Initializing all the trading strategies and observing individual strategy performance
sma_lag1, sma_lag2 = 7, 14
ewm_a1, ewm_a2 = 0.1, 0.2

ewm = trading_strategies.ewm_trend_following(trd_input,ewm_a1,ewm_a2,'ewm')
sma = trading_strategies.sma_trend_following(trd_input,sma_lag1,sma_lag2,'sma')

ax=px.line(y=[sma.cum_return_strat,ewm.cum_return_strat,sma.cum_return_buy_n_hold],
           x=sma.index)
ax.update_layout(yaxis=dict(range=[0, 20]))
ax.show()

In [259]:
sma['signal'].value_counts()
sma

sma,ESc1,return,log,log_return,hmm,SMA_7,SMA_14,signal,action,log_return_buy_n_hold,log_return_strat,cum_return_buy_n_hold,cum_return_strat
date,,,,,,,,,,,,,
2006-01-03,1280.50,1.004511,7.155006,0.004501,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2006-01-04,1281.25,1.000586,7.155591,0.000586,1,NaN,NaN,0,0.0,0.000586,0.000000,1.000586,1.000000
2006-01-05,1291.75,1.008195,7.163753,0.008162,1,NaN,NaN,0,0.0,0.008162,0.000000,1.008786,1.000000
2006-01-08,1295.00,1.002516,7.166266,0.002513,1,NaN,NaN,0,0.0,0.002513,0.000000,1.011324,1.000000
2006-01-09,1296.00,1.000772,7.167038,0.000772,1,NaN,NaN,0,0.0,0.000772,0.000000,1.012105,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,4324.25,0.999711,8.371994,-0.000289,0,4336.464286,4409.732143,0,0.0,-0.000289,-0.000000,3.377001,6.183488
2023-10-02,4264.75,0.986240,8.358139,-0.013855,0,4322.714286,4391.678571,0,0.0,-0.013855,-0.000000,3.330535,6.183488
2023-10-03,4297.75,1.007738,8.365847,0.007708,0,4311.142857,4373.303571,0,0.0,0.007708,0.000000,3.356306,6.183488


In [244]:
# Calculating portfolio returns

def calculate_portfolio_returns(s1,s2):
    portfolio = pd.DataFrame()
    portfolio.columns.name = 'portfolio'
    portfolio['bnh_cum_log_returns'] = s1['cum_return_buy_n_hold']
    portfolio[f'{s1.columns.name}_cum_log_returns'] = s1['cum_return_strat']
    portfolio[f'{s2.columns.name}_cum_log_returns'] = s2['cum_return_strat']

    strat_num = portfolio.shape[1]-1
    weight = 1/strat_num

    portfolio['port_cum_log_returns'] = weight * portfolio.iloc[:, 1:strat_num+1].sum(axis=1)

    return portfolio

portfolio = calculate_portfolio_returns(sma,ewm)
portfolio.drop(portfolio.index[0],inplace=True)
portfolio

portfolio,bnh_cum_log_returns,sma_cum_log_returns,ewm_cum_log_returns,port_cum_log_returns
date,,,,
2006-01-04,1.000586,1.000000,1.000000,1.000000
2006-01-05,1.008786,1.000000,1.000000,1.000000
2006-01-08,1.011324,1.000000,1.000000,1.000000
2006-01-09,1.012105,1.000000,1.000000,1.000000
2006-01-10,1.015033,1.000000,1.000000,1.000000
...,...,...,...,...
2023-10-01,3.377001,9.820110,34.838749,22.329430
2023-10-02,3.330535,9.957116,35.324805,22.640961
2023-10-03,3.356306,9.880661,35.053566,22.467113


Implementing regime signals derived from HMM

In [271]:
# Evaluating each strategy based on the hmm signals

sma_hmm_1 = trading_strategies.sma_trend_following(trd_input,sma_lag1,sma_lag2,'sma_hmm_1',hmm=True,regime=1)
sma_hmm_0 = trading_strategies.sma_trend_following(trd_input,sma_lag1,sma_lag2,'sma_hmm_0',hmm=True,regime=0)

ewm_hmm_1 = trading_strategies.ewm_trend_following(trd_input,ewm_a1,ewm_a2,'ewm_hmm_1',hmm=True,regime=1)
ewm_hmm_0 = trading_strategies.ewm_trend_following(trd_input,ewm_a1,ewm_a2,'ewm_hmm_0',hmm=True,regime=0)

hmm_all = pd.concat([sma['cum_return_strat'],sma_hmm_1['cum_return_strat'],sma_hmm_0['cum_return_strat'],ewm['cum_return_strat'],ewm_hmm_1['cum_return_strat'],ewm_hmm_0['cum_return_strat']],axis=1)
hmm_all.columns = ['sma','sma_hmm_1', 'sma_hmm_0','ewm', 'ewm_hmm_1', 'ewm_hmm_0']
hmm_all

,sma,sma_hmm_1,sma_hmm_0,ewm,ewm_hmm_1,ewm_hmm_0
date,,,,,,
2006-01-03,NaN,NaN,NaN,1.000000,1.000000,1.000000
2006-01-04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2006-01-05,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2006-01-08,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2006-01-09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...
2023-10-01,9.820110,6.183488,1.588118,34.838749,21.669633,1.356564
2023-10-02,9.957116,6.183488,1.610275,35.324805,21.669633,1.375491
2023-10-03,9.880661,6.183488,1.597911,35.053566,21.669633,1.364929


In [287]:
# Evaluation should be some rigorous, actual method. Calculate sharpe, sortino, volatility of returns, max drawdown, number of trades, trade frequency 
# Compare over periods of time

def calculate_performance(strat, rf):
    # Calculating Sharpe
    returns = np.exp(strat['log_return_strat'])
    print(returns)
    annualized_return = (1 + returns.mean())**(252-1)
    annualized_vol = returns.std()*np.sqrt(252)
    sharpe = (annualized_return - rf)/annualized_vol

    # Calculating Sortino
    downside_returns = returns[returns < 0]
    annualized_down_vol = downside_returns.std() * np.sqrt(252)
    sortino = (annualized_return - rf)/annualized_down_vol

    return sharpe, sortino

s_sma_hmm_1,sortino = calculate_performance(sma,0.03)
print(s_sma_hmm_1,sortino)


date
2006-01-03         NaN
2006-01-04    1.000000
2006-01-05    1.000000
2006-01-08    1.000000
2006-01-09    1.000000
                ...   
2023-10-01    1.000289
2023-10-02    1.013952
2023-10-03    0.992322
2023-10-04    1.001631
2023-10-05    0.988310
Name: log_return_strat, Length: 4488, dtype: float64
1.9571676969550565e+76 nan


In [ ]:


# Make and adjusted version of the strategies that only run during the regimes where they perform the best in

# Compare portfolio performances


Implementing an investment strategy

In [234]:
prices_with_states = pd.DataFrame(prices[split_index:][f'{trading_instrument}'])
prices_with_states['State'] = states_pred_hmm
prices_with_states.head()

,ESc1,State
date,,
2006-01-02,1274.75,1
2006-01-03,1280.50,1
2006-01-04,1281.25,1
2006-01-05,1291.75,1
2006-01-08,1295.00,1


In [235]:
prices_with_states['P&L_daily'] = np.log(prices_with_states['ESc1'] / prices_with_states['ESc1'].shift(1)).dropna()
prices_with_states.head()

,ESc1,State,P&L_daily
date,,,
2006-01-02,1274.75,1,NaN
2006-01-03,1280.50,1,0.004501
2006-01-04,1281.25,1,0.000586
2006-01-05,1291.75,1,0.008162
2006-01-08,1295.00,1,0.002513


In [236]:
prices_with_states['State'] = prices_with_states['State'].shift(1)
prices_with_states.dropna(inplace = True)

In [237]:
prices_with_states['Position'] = np.where(prices_with_states['State'] == 1,1,-1)
prices_with_states.head()

,ESc1,State,P&L_daily,Position
date,,,,
2006-01-03,1280.50,1.0,0.004501,1
2006-01-04,1281.25,1.0,0.000586,1
2006-01-05,1291.75,1.0,0.008162,1
2006-01-08,1295.00,1.0,0.002513,1
2006-01-09,1296.00,1.0,0.000772,1


In [238]:
prices_with_states['Daily_Outcome_hmm'] = prices_with_states['Position'] * prices_with_states['P&L_daily']
prices_with_states['Cumulative_Outcome_BaH'] = prices_with_states['P&L_daily'].cumsum()
prices_with_states['Cumulative_Outcome_hmm'] = prices_with_states['Daily_Outcome_hmm'].cumsum()
prices_with_states

,ESc1,State,P&L_daily,Position,Daily_Outcome_hmm,Cumulative_Outcome_BaH,Cumulative_Outcome_hmm
date,,,,,,,
2006-01-03,1280.50,1.0,0.004501,1,0.004501,0.004501,0.004501
2006-01-04,1281.25,1.0,0.000586,1,0.000586,0.005086,0.005086
2006-01-05,1291.75,1.0,0.008162,1,0.008162,0.013248,0.013248
2006-01-08,1295.00,1.0,0.002513,1,0.002513,0.015761,0.015761
2006-01-09,1296.00,1.0,0.000772,1,0.000772,0.016533,0.016533
...,...,...,...,...,...,...,...
2023-10-01,4324.25,0.0,-0.000289,-1,0.000289,1.221489,1.153233
2023-10-02,4264.75,0.0,-0.013855,-1,0.013855,1.207633,1.167088
2023-10-03,4297.75,0.0,0.007708,-1,-0.007708,1.215342,1.159380


In [239]:
fig = go.Figure()
 
fig.add_trace(go.Line(x=prices_with_states.index, y=prices_with_states["Cumulative_Outcome_BaH"], 
                      name = 'Cumulative_Outcome_BaH', line_color = 'navy'))
 
fig.add_trace(go.Line(x= prices_with_states.index, y=prices_with_states['Cumulative_Outcome_hmm'], 
                      name = 'Cumulative_Outcome_hmm', line_color = 'olive'))
 
fig.update_layout(height=400, width=900, legend=dict(
    yanchor="top", y=0.99, xanchor="left",x=0.01), 
    margin=dict(l=20, r=20, t=20, b=20))
 
fig.show()